In [ ]:
import requests
import folium
import json
import pystac

In [ ]:
m = folium.Map(
    location=(47.6062, -122.3321),
    tiles="cartodbpositron",
    zoom_start=4,
)

snotel_stations = "https://raw.githubusercontent.com/scottyhq/snotel/main/snotel-sites.geojson"
folium.GeoJson(snotel_stations, embed=False, name="snotel").add_to(m)

# Add a Planetary Computer Layer
# Docs: https://planetarycomputer.microsoft.com/docs/quickstarts/using-the-data-api/

# Get this url from Planetary computer Explorer
item_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/S1A_IW_GRDH_1SDV_20230111T234137_20230111T234202_046743_059A8C_rtc"
# Add to Map (PC data API takes care of dynamic tiling)
item = pystac.Item.from_file(item_url)
r = requests.get(item.assets["tilejson"].href).json()
tiles = r["tiles"][0]
folium.TileLayer(tiles=tiles, overlay=True, name='S1RTC', attr="Planetary Computer").add_to(m)

# Add a public COG custom TIF layer using TiTiler (thanks devseed!)
tif = "https://sentinel-s1-rtc-indigo.s3.us-west-2.amazonaws.com/tiles/RTC/1/IW/12/S/YJ/2016/S1B_20161121_12SYJ_ASC/Gamma0_VV.tif"
tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
expression = "expression=sqrt(b1)&rescale=0,1"
virtual_tiles = f"{tiler}?url={tif}&{expression}"
folium.TileLayer(tiles=virtual_tiles, overlay=True, name='12SYJ', attr="IndigoAg").add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("index.html")